In [1]:
from kafka import KafkaProducer
import json
import time
import uuid
import random
import math
from datetime import datetime

# --------------------------
# 1) INITIAL SETUP
# --------------------------
# List of farms
farms = [
    {"farm_id": "farm_1", "region": "NileDelta", "crop_type": "Wheat"},
    {"farm_id": "farm_2", "region": "NileDelta", "crop_type": "Rice"},
    {"farm_id": "farm_3", "region": "NileDelta", "crop_type": "Onion"},
    {"farm_id": "farm_4", "region": "UpperEgypt", "crop_type": "Tomato"},
    {"farm_id": "farm_5", "region": "UpperEgypt", "crop_type": "Dates"},
    {"farm_id": "farm_6", "region": "UpperEgypt", "crop_type": "Peanuts"},
    {"farm_id": "farm_7", "region": "Sinai", "crop_type": "Corn"},
    {"farm_id": "farm_8", "region": "Sinai", "crop_type": "Olive"},
    {"farm_id": "farm_9", "region": "Sinai", "crop_type": "Barley"},
    {"farm_id": "farm_10", "region": "Sinai", "crop_type": "Potato"},
]

# Initial values for each farm
farm_states = {
    f["farm_id"]: {
        "soil_moisture": random.uniform(30, 45),
        "soil_pH": random.uniform(6.0, 7.0),
        "temperature": random.uniform(20, 30)
    }
    for f in farms
}

midpoints = {
    "soil_moisture": 40,
    "temperature": 25,
    "humidity": 55,
    "soil_pH": 6.5
}

def generate_record(farm, state):
    """Generate ONE new sensor reading."""
    now = datetime.now()
    hour = now.hour
    day = now.timetuple().tm_yday

    # Sunlight model
    if 6 <= hour <= 18:
        base = 12 * math.exp(-((hour - 12) ** 2) / 18)
        sunlight = base * (0.8 + 0.2 * math.sin((2 * math.pi * day) / 365))
    else:
        sunlight = 0

    # Temperature drift
    seasonal = 5 * math.sin((2 * math.pi * day) / 365)
    target_temp = 18 + 1.4 * sunlight + seasonal
    state["temperature"] += (target_temp - state["temperature"]) * 0.3
    state["temperature"] += random.uniform(-0.3, 0.3)

    # Rainfall
    rainfall = random.uniform(10, 80) if random.random() < 0.014 else 0

    # Soil moisture
    state["soil_moisture"] -= 0.15 * (state["temperature"] / 25)
    state["soil_moisture"] += random.uniform(-0.3, 0.3)
    if rainfall > 0:
        state["soil_moisture"] += 0.6 * rainfall
    if state["soil_moisture"] < 20:
        state["soil_moisture"] += random.uniform(20, 30)
    state["soil_moisture"] = max(0, min(100, state["soil_moisture"]))

    # Humidity
    humidity = 55 - 0.8 * (state["temperature"] - 20) + 0.25 * rainfall + random.uniform(-3, 3)
    humidity = max(10, min(100, humidity))

    # Soil pH
    state["soil_pH"] += random.uniform(-0.02, 0.02)
    if rainfall > 50: state["soil_pH"] -= 0.2
    elif rainfall > 0: state["soil_pH"] -= 0.1
    if state["soil_moisture"] > 60: state["soil_pH"] += 0.1

    # Pesticide
    pesticide = 0
    if (rainfall > 0 and random.random() < 0.7) or (hour % (24 * 7) == 0):
        pesticide = round(random.uniform(5, 20), 2)
        state["soil_pH"] -= 0.2

    # Stability correction
    if abs(state["temperature"] - midpoints["temperature"]) > 15:
        state["temperature"] = 0.7 * state["temperature"] + 0.3 * midpoints["temperature"]

    if state["soil_moisture"] < 10 or state["soil_moisture"] > 90:
        state["soil_moisture"] += 0.4 * (midpoints["soil_moisture"] - state["soil_moisture"])

    if humidity < 15 or humidity > 90:
        humidity += 0.4 * (midpoints["humidity"] - humidity)

    if state["soil_pH"] < 5.0 or state["soil_pH"] > 8.0:
        state["soil_pH"] += 0.4 * (midpoints["soil_pH"] - state["soil_pH"])

    record = {
        "sensor_id": str(uuid.uuid4()),
        "timestamp": now.isoformat(),
        "soil_moisture": round(state["soil_moisture"], 2),
        "soil_pH": round(state["soil_pH"], 2),
        "temperature": round(state["temperature"], 2),
        "rainfall": round(rainfall, 2),
        "humidity": round(humidity, 2),
        "sunlight_intensity": round(sunlight, 2),
        "pesticide_usage_ml": pesticide,
        "farm_id": farm["farm_id"],
        "region": farm["region"],
        "crop_type": farm["crop_type"]
    }

    return record

print("✅ Generator setup complete. Farms loaded, states initialized, and record generator is ready.")

✅ Generator setup complete. Farms loaded, states initialized, and record generator is ready.


In [3]:
# --------------------------
# 2) STREAM FOREVER
# --------------------------
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

topic = "smart_farming_data"

print("🚀 Real-time generator started... sending one record every 3 seconds.\n")

while True:
    for farm in farms:
        state = farm_states[farm["farm_id"]]
        record = generate_record(farm, state)

        producer.send(topic, value=record)
        producer.flush()

        print("📤 Sent:", record)

        time.sleep(1)

🚀 Real-time generator started... sending one record every 3 seconds.

📤 Sent: {'sensor_id': '1b072f70-3c4e-4b91-b9f1-0144ae3df4b7', 'timestamp': '2025-11-26T22:10:07.006285', 'soil_moisture': 40.09, 'soil_pH': 6.43, 'temperature': 20.43, 'rainfall': 0, 'humidity': 55.03, 'sunlight_intensity': 0, 'pesticide_usage_ml': 0, 'farm_id': 'farm_1', 'region': 'NileDelta', 'crop_type': 'Wheat'}
📤 Sent: {'sensor_id': '11ee3ff9-a8cb-4b63-886c-9097a8c2d48a', 'timestamp': '2025-11-26T22:10:08.662812', 'soil_moisture': 40.06, 'soil_pH': 6.36, 'temperature': 22.36, 'rainfall': 0, 'humidity': 51.54, 'sunlight_intensity': 0, 'pesticide_usage_ml': 0, 'farm_id': 'farm_2', 'region': 'NileDelta', 'crop_type': 'Rice'}
📤 Sent: {'sensor_id': '18235bee-d115-40f4-af65-d69cd82be18b', 'timestamp': '2025-11-26T22:10:09.681076', 'soil_moisture': 35.11, 'soil_pH': 6.74, 'temperature': 19.77, 'rainfall': 0, 'humidity': 56.82, 'sunlight_intensity': 0, 'pesticide_usage_ml': 0, 'farm_id': 'farm_3', 'region': 'NileDelta',

KeyboardInterrupt: 